# Using NER to Tag Data

We are using the dataset pulled from */r/investing* and the SpaCy NER code covered in the previous sections.

In [1]:
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_sm')

The NER code will be reformated into a function that will consume a length of text, and return a list of mentioned organizations:

In [2]:
def get_orgs(text):
    # process the text with our SpaCy model to get named entities
    doc = nlp(text)
    # initialize list to store identified organizations
    org_list = []
    # loop through the identified entities and append ORG entities to org_list
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

## Applying NER

All we need to do now is load in the */r/investing* data and apply the `get_orgs` function to our text column to create a new `organizations` column.

1. Load the data and view the top five rows with `df.head()`:

In [3]:
df = pd.read_csv('./data/reddit_investing.csv', sep='|')
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,t3_wg6ntw,1.659633e+09,investing,Is this the end of the traditional stock market?,Please consider the following points from a pe...,0.37,0.0,0.0,0.0
1,t3_wg6bl9,1.659632e+09,investing,NERV is going nuts. Are my options worthless?,What happens to options when they are no longe...,0.50,0.0,0.0,0.0
2,t3_wg29p4,1.659622e+09,investing,Earnings boost Toronto stocks,[https://www.reuters.com/markets/stocks/tsx-fu...,0.81,82.0,0.0,82.0
3,t3_wg0os3,1.659618e+09,investing,Convertible bond and nuke,Can someone please explain to me what this “nu...,0.85,41.0,0.0,41.0
4,t3_wfw9yh,1.659604e+09,investing,Daily General Discussion and Advice Thread - A...,Have a general question? Want to offer some c...,0.86,22.0,0.0,22.0


2. Extract mentioned organizations from `selftext` and add to a new column called `organizations`:

In [4]:
df['selftext'].apply(get_orgs)

0                                                     []
1                          [Minerva Neurosciences, NERV]
2      [Canadian Natural Resources, the Toronto Stock...
3                                                     []
4                                                  [FAQ]
                             ...                        
890                                               [XEQT]
891                                           [treasury]
892                                               [Ally]
893                                                   []
894                                             [Buffet]
Name: selftext, Length: 895, dtype: object

In [5]:
df['organizations'] = df['selftext'].apply(get_orgs)
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations
0,t3_wg6ntw,1.659633e+09,investing,Is this the end of the traditional stock market?,Please consider the following points from a pe...,0.37,0.0,0.0,0.0,[]
1,t3_wg6bl9,1.659632e+09,investing,NERV is going nuts. Are my options worthless?,What happens to options when they are no longe...,0.50,0.0,0.0,0.0,"[Minerva Neurosciences, NERV]"
2,t3_wg29p4,1.659622e+09,investing,Earnings boost Toronto stocks,[https://www.reuters.com/markets/stocks/tsx-fu...,0.81,82.0,0.0,82.0,"[Canadian Natural Resources, the Toronto Stock..."
3,t3_wg0os3,1.659618e+09,investing,Convertible bond and nuke,Can someone please explain to me what this “nu...,0.85,41.0,0.0,41.0,[]
4,t3_wfw9yh,1.659604e+09,investing,Daily General Discussion and Advice Thread - A...,Have a general question? Want to offer some c...,0.86,22.0,0.0,22.0,[FAQ]


*(This step can take a long time to run. It can be useful to break larger datasets into more manageable chunks if required)*

Now we have a list of all mentioned organizations contained within the `organizations` column. We will now take the full column, merge each list, and use `Counter` to create a frequency table of organization mentions.

In [6]:
# merge organizations column into one big list
orgs = df['organizations'].to_list()
orgs = [org for sublist in orgs for org in sublist]
orgs[:10]

['Minerva Neurosciences',
 'NERV',
 'Canadian Natural Resources',
 "the Toronto Stock Exchange's",
 'Bombardier',
 '.GSPTSE)**](https://www.reuters.com/quote/.GSPTSE',
 'Reuters',
 'FAQ',
 'EPS',
 'LOC']

In [7]:
from collections import Counter

In [8]:
# create dictionary of organization mention frequency
org_freq = Counter(orgs)

We now have a *Counter* dictionary containing all of our organization labels as *keys*, and their mention frequency as *values*. The `most_common(n)` method allows us to view the **n** most frequently mentioned organizations:

In [9]:
org_freq.most_common(10)

[('FAQ', 70),
 ('ETF', 52),
 ('Fed', 40),
 ('Reuters', 19),
 ('VOO', 17),
 ('Treasury', 13),
 ('DCA', 13),
 ('EPS', 12),
 ('Amazon', 12),
 ('treasury', 12)]

Clearly there is a need to do some further pruning of the data to remove non-organization labels like *EV* (electric vehicle), *COVID*, *ETF* (exchange trading fund), *NYSE*, and *SPAC* (special acquisition company). Depending on the use-case it may even be useful to keep a few of these, or remove a few others like *SEC* and *FDA*.

To do this, we would create a custom list and implement it in our `get_orgs` function like so:

In [10]:
BLACKLIST = ['ev', 'covid', 'etf', 'nyse', 'sec', 'spac', 'fda']
BLACKLIST += ["faq", ]

def get_orgs(text):
    doc = nlp(text)
    org_list = []
    for entity in doc.ents:
        # here we modify the original code to check that entity text is not equal to one of our 'blacklisted' organizations
        # (we also add .lower() to lowercase the text, this allows us to match both 'nyse' and 'NYSE' with just 'nyse')
        if entity.label_ == 'ORG' and entity.text.lower() not in BLACKLIST:
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

If we then apply this new `get_org` function to our dataframe we will see that *ETF* that originally appeared alongside *ARK* in the second row is no longer included:

In [11]:
df['organizations'] = df['selftext'].apply(get_orgs)
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations
0,t3_wg6ntw,1.659633e+09,investing,Is this the end of the traditional stock market?,Please consider the following points from a pe...,0.37,0.0,0.0,0.0,[]
1,t3_wg6bl9,1.659632e+09,investing,NERV is going nuts. Are my options worthless?,What happens to options when they are no longe...,0.50,0.0,0.0,0.0,"[Minerva Neurosciences, NERV]"
2,t3_wg29p4,1.659622e+09,investing,Earnings boost Toronto stocks,[https://www.reuters.com/markets/stocks/tsx-fu...,0.81,82.0,0.0,82.0,"[Canadian Natural Resources, the Toronto Stock..."
3,t3_wg0os3,1.659618e+09,investing,Convertible bond and nuke,Can someone please explain to me what this “nu...,0.85,41.0,0.0,41.0,[]
4,t3_wfw9yh,1.659604e+09,investing,Daily General Discussion and Advice Thread - A...,Have a general question? Want to offer some c...,0.86,22.0,0.0,22.0,[]


We can save our dataframe with the new *organizations* column to file for later use.

In [12]:
df.to_csv('./data/processed/reddit_investing_ner.csv', sep='|', index=False)